# scaredy rat uses raw data from EthoVision to analyze behavior over time. 

#### To run, change variables in specified cell, then go to Kernel > Restart & Run All
Known possible issues:
 - for whatever reason, the raw data column for Tone 10 doesn't have a space (i.e. "Tone10" instead of "Tone 10). fix this by ctrl-F-ing the raw data and making that replacement. it will throw an error if left unchanged.
 - there won't be data for the 12th tone due to indexing issues (sometimes recording ends before completing time bins). fix tbd. for now, it just ends at Tone 11.
 - various things need to be changed if there is a different number of tones - email.
 - in the **animal_read** function, the indexes are specific to the ethovision output. sometimes there are other rows in the header, so the numbers will have to be changed. if that function throws an error, this may be the issue. for reference, the animal ID is row 33 on the excel file, so the corresponding index is **df.iloc[31]**. if the animal ID were on row 34, you would change the index to **df.iloc[32]**.

for others, email me at annajxli@gmail.com

In [ ]:
%matplotlib inline

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML
import math
import time

In [ ]:
def animal_read(filename,sheet):
    '''
    Returns corresponding animals for sheets
    Reads data into dataframe with long header removed.
    Uses correct labels as column labels.
    Uses time as row indexes.
    '''
    filepath = os.path.join(inpath,filename)
    df = pd.read_excel(filepath,sheetname=sheet,skip_footer=34,index_col=0)
    animal = df.iloc[31][0]
    print(filename + " " + sheet + " is " + animal)
    
    df = pd.read_excel(filepath,sheetname=sheet,skiprows=34,index_col=0,headers=0)
    df = df[1:]
    df.replace(to_replace='-',value=0,inplace=True)
    return(animal,df)

In [ ]:
def find_tones(df):
    '''
    Creates dictionary of each tone. Values are
    dataframes for times at which each tone == 1.
    '''
    tones = {}
    i = 1
    while i <= 12: # number of tones
        num = str(i)
        label = 'Tone ' + num
        tone = pd.DataFrame(df[df[label] == 1])
        tones[i] = tone
        i += 1
    return(tones)

In [ ]:
def find_pretones(df):
    '''
    Creates dictionary of each pretone. Values
    are dataframes for 30s before tone == 1
    '''
    pretones = {}
    i = 1
    while i<= 12:
        num = str(i)
        label = 'Tone ' + num #Column label for each tone
        
        tone = df[df[label] == 1] #Tone dataframe
        tonestart = tone.iloc[0] #Time for tone start
        
        starttime = math.floor(tonestart['Recording time']-30) #Time for pretone start
        endtime = math.floor(tonestart['Recording time']) #Time for pretone end
        itime = df.index.get_loc(starttime,method='bfill') #Index for pretone start
        etime = df.index.get_loc(endtime,method='bfill') #Index for pretone end
        
        pretone = df.iloc[itime:etime] #df for pretone
        pretones[i] = pretone #dictionary for all pretones
        i += 1
    return(pretones)

In [ ]:
def find_shock_responses(df):
    '''
    Creates dictionary of each shock response. Values
    are dataframes for 5s after tone == 1
    '''
    sresponses = {}
    i = 1
    while i<= 11:
        num = str(i)
        label = 'Tone ' + num #Column label for each tone
        
        tone = df[df[label] == 1] #Tone dataframe
        toneend = tone.iloc[-1] #Time for tone end
        
        starttime = math.floor(toneend['Recording time']) #Time for sresponse start
        endtime = math.floor(toneend['Recording time'] + 5) #Time for sresponse end
        itime = df.index.get_loc(starttime,method='bfill') #Index for sresponse start
        etime = df.index.get_loc(endtime,method='bfill') #Index for sresponse end
        
        sresponse = df.iloc[itime:etime] #df for sresponse
        sresponses[i] = sresponse #dictionary for all sresponse
        i += 1
    return(sresponses)

In [ ]:
def find_postshocks(df):
    '''
    Creates dictionary of each postshock. Values
    are dataframes for 5s after tone == 1
    '''
    pshocks = {}
    i = 1
    while i<= 11:
        num = str(i)
        label = 'Tone ' + num #Column label for each tone
        
        tone = df[df[label] == 1] #Tone dataframe
        toneend = tone.iloc[-1] #Time for tone end
        
        starttime = math.floor(toneend['Recording time'] + 5) #Time for pshock start
        endtime = math.floor(toneend['Recording time'] + 35) #Time for pshock end
        itime = df.index.get_loc(starttime,method='bfill') #Index for pshock start
        etime = df.index.get_loc(endtime,method='bfill') #Index for pshock end
        
        pshock = df.iloc[itime:etime] #df for pshock
        pshocks[i] = pshock #dictionary for all sresponse
        i += 1
    return(pshocks)

In [ ]:
def get_total_dists(datadict):
    i = 1
    while i <= 12:
        epoch = datadict[i]
        dist = epoch['Distance moved']
        total = dist.sum()
        print()

In [ ]:
def get_means(datadict,timebin):
    meanlist = []
    i = 1
    while i <= 11:
        epoch = datadict[i]
        vels = epoch['Velocity']
        mean = round(vels.mean(),3)
        meanlist.append(mean)
        i += 1
    means = pd.DataFrame(meanlist,columns=[timebin + ' Mean Velocity'])
    means.index = np.arange(1, len(means) + 1)
    return(means)

In [ ]:
def get_meds(datadict,timebin):
    medlist = []
    i = 1
    while i <= 11:
        epoch = datadict[i]
        vels = epoch['Velocity']
        med = round(vels.median(),3)
        medlist.append(med)
        i += 1
    meds = pd.DataFrame(medlist,columns=[timebin + ' Median Velocity'])
    meds.index = np.arange(1, len(meds) + 1)
    return(meds)

In [ ]:
def get_vels(df):
    '''
    Creates data of all velocities from
    original dataframe for plotting.
    '''
    tonevels = {}
    i = 1
    while i <= 12: # number of tones
        vels = []
        num = str(i)
        label = 'Tone ' + num
        tone = pd.DataFrame(df[df[label] == 1])
        vels.append(tone['Velocity'])
        tonevels[i] = vels
        i += 1
    return(tonevels)

# **Change variables in this cell only**

In [ ]:
## This is the folder with the raw data
## Be sure to put r before the path
## Easiest to go to correct folder, then copy/paste its path
inpath = r'C:\Users\Anna\Documents\Shansky Lab\BRF Darting\BRF Anna data\FC'

## This is where the output goes
## Make sure the folder exists already.
## The file does not need to exist; it will be created.
outpath = r'C:\Users\Anna\Documents\Shansky Lab\BRF Darting\scaredy rat results'

filename = 'Raw data-BRF Fear Conditioning-Trial     1.xlsx' ## include .xlsx
sheetname = 'Track-Arena 4-Subject 1'

## These are function calls
#### Take a while to run due to size of data sheet

In [ ]:
ID,anim = animal_read(filename,sheetname)

tones = find_tones(anim)
pretones = find_pretones(anim)
shocks = find_shock_responses(anim)
pshocks = find_postshocks(anim)

tmeans = get_means(tones,'Tone')
ptmeans = get_means(pretones,'Pre-tone')
smeans = get_means(shocks,'Shock')
pshockmeans = get_means(pshocks,'Post-shock')
allmeans = pd.concat([tmeans, ptmeans, smeans, pshockmeans],axis=1)

tmeds = get_meds(tones,'Tone')
ptmeds = get_meds(pretones,'Pre-tone')
smeds = get_meds(shocks,'Shock')
pshockmeds = get_meds(pshocks,'Post-shock')
allmeds = pd.concat([tmeds, ptmeds, smeds, pshockmeds],axis=1)

## Output data and figures will be under this cell
#### Figures will also be saved

In [ ]:
display(allmeans)
display(allmeds)

## This cell saves the mean/med velocities

In [ ]:
## Saves the data to a file named with current date.
i = time.strftime("%m.%d.%Y")
meanoutfile = outpath + '\\scaredy-rat-mean-' + ID + '-' + i + '.csv'
medoutfile = outpath + '\\scaredy-rat-med-' + ID + '-' + i + '.csv'

allmeans.to_csv(meanoutfile)
allmeds.to_csv(medoutfile)

#### future implementations:
 - "known issues"
 - conditional on whether or not data for last one is present (last tone thing)
 - automatically detect tone num (probably easiest? idk)
 - findpeaks_cwt maybe (from signal)
 - nested functions

# Test section
### (Code that may or may not work yet)